# Flying Experiments 

Detect and localize the wall while flying and playing sweeps.

In [ ]:
import math
import sys

import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 14
rcParams["text.usetex"] = True

In [ ]:
# check that interpolation works as expected
from numpy import array, float32
freqs =  array([2500., 2562., 2625., 2687., 2750., 2812., 2875., 2937., 3000.,
       3062., 3125., 3187., 3250., 3312., 3375., 3437., 3562., 3625.,
       3687., 3750., 3812., 3875., 3937., 4000., 4062., 4125., 4187.,
       4250., 4312., 4375., 4437., 4500.], dtype=float32) 
freqs_interp =  array([2500., 2562., 2625., 2687., 2750., 2812., 2875., 2937., 3000.,
       3062., 3125., 3187., 3250., 3312., 3375., 3437., 3499., 3562.,
       3625., 3687., 3750., 3812., 3875., 3937., 4000., 4062., 4125.,
       4187., 4250., 4312., 4375., 4437., 4500.], dtype=float32)

plt.figure()
plt.scatter(freqs_interp, [0.5] * len(freqs_interp), label="interpolated")
plt.scatter(freqs, [0.6] * len(freqs), label="original")
plt.legend()
plt.ylim(0, 1)

freqs =  array([3140., 3187., 3234., 3328., 3375., 3468., 3515., 3562., 3984.,
       4031., 4078., 4171., 4218., 4265., 4359., 4406., 4500., 4546.],
      dtype=float32)
freqs_interp =  array([3140., 3187., 3234., 3280., 3328., 3375., 3421., 3468., 3515.,
       3562., 3984., 4031., 4078., 4124., 4171., 4218., 4265., 4311.,
       4359., 4406., 4452., 4500., 4546.], dtype=float32) 

plt.figure()
plt.scatter(freqs_interp, [0.5] * len(freqs_interp), label="interpolated")
plt.scatter(freqs, [0.6] * len(freqs), label="original")
plt.legend()
plt.ylim(0, 1)

freqs =  array([1156., 1281., 1406., 1546., 1671., 1796., 1921., 2062., 2187.,
       2312., 2437., 2562., 2718., 2843., 2953., 3109., 3234., 3359.,
       3468., 3625., 3734., 3859., 3984., 4125., 4265., 4421., 4515.,
       4687., 4781., 4875., 5078., 5187.], dtype=float32) 
freqs_interp =  array([1156., 1281., 1406., 1546., 1671., 1796., 1921., 2062., 2187.,
       2312., 2437., 2562., 2718., 2843., 2953., 3109., 3234., 3359.,
       3468., 3625., 3734., 3859., 3984., 4125., 4265., 4421., 4515.,
       4687., 4781., 4875., 5000., 5078., 5187.], dtype=float32)


plt.figure()
plt.scatter(freqs_interp, [0.5] * len(freqs_interp), label="interpolated")
plt.scatter(freqs, [0.6] * len(freqs), label="original")
plt.legend()
plt.ylim(0, 1)

In [ ]:
def get_title(row, ignore=[]):
    categories = row.index.drop(["matrix distances", "matrix angles", "distances_cm", "angles_deg", "average_time"]).values
    title = ''
    for param_name in row.index:
        if param_name in ignore:
            continue
        if param_name in categories:
            param_value = row[param_name]
            if type(param_value) in (float, np.float64, np.float32):
                param_value = round(param_value, 1)
            title += f'{param_name}: {param_value}, '
    return title

In [ ]:
def get_estimates_here(results_matrix, xvalues, method="max"):
    if method in ["argmax", "max"]:
        estimates = xvalues[np.argmax(results_matrix, axis=0)].astype(float)
    elif method == "mean":
        estimates = np.empty(results_matrix.shape[1])
        for col in range(results_matrix.shape[1]):
            mean = np.average(xvalues, weights=results_matrix[:, col], axis=0)
            estimates[col] = mean
    else:
        raise ValueError(method)
    valid = np.any(results_matrix > 0, axis=0)
    estimates[~valid] = np.nan
    return estimates

In [ ]:
def plot_matrix(yvalues, results_matrix, gt_values=None, 
                xvalues=None, no_deco=False, angles=False, 
                log=True, n_calib=0):
    from utils.plotting_tools import add_colorbar, pcolorfast_custom, FIGSIZE
    from copy import copy
    
    cmap = copy(plt.get_cmap())
    cmap.set_bad('gray')
    if xvalues is None:
        xvalues = np.arange(results_matrix.shape[1])
        xlabel = "index [-]"
    else:
        xlabel = "time [s]"
        
    estimates = get_estimates_here(results_matrix, yvalues)
    fig, ax = plt.subplots()
    fig.set_size_inches(2 * FIGSIZE, FIGSIZE)
    
    if log:
        plot_matrix = copy(results_matrix)
        plot_matrix[plot_matrix == 0] = np.nan
        plot_matrix[plot_matrix > 0] = np.log10(plot_matrix[plot_matrix>0])
    else:
        plot_matrix = results_matrix
        
    im = ax.pcolormesh(xvalues, yvalues, plot_matrix, cmap=cmap)
    
    if gt_values is not None:
        ax.plot(
            xvalues,
            gt_values,
            color="black",
            label="\\textbf{ground truth}",
            marker='x'
        )
    ax.set_ylim(min(yvalues), max(yvalues))
    ax.plot(
        xvalues,
        estimates,
        color="white",
        label="\\textbf{estimates}",
        marker='o',
        ls='-'
    )
    if no_deco:
        ax.set_yticks([])
        ax.set_xticks([])
    else:
        add_colorbar(fig, ax, im, title="log-probability" if log else "probability")
        ax.set_ylabel("estimated angle [deg]" if angles else "estimated distance [cm]")
        ax.set_xlabel(xlabel)
        
        leg = ax.legend(framealpha=0, loc='upper right')
        for text in leg.get_texts():
            plt.setp(text, color='w', weight='bold')
        #ax.set_xticks(xticks, labels=xticklabels)
        #ax.set_yticks(yticks, labels=yticks)
        #n_xticks = 5
        #step = len(xvalues) // n_xticks
        #ax.set_xticks(np.round(xvalues[::step], 1))
        #ax.set_xticklabels(np.round(xvalues[::step], 1))
        
    alpha = 0.7
    if n_calib:
        from matplotlib.patches import Rectangle
        width = xvalues[n_calib] - xvalues[0]
        height= max(yvalues) - min(yvalues)
        diff_x = xvalues[1] - xvalues[0]
        xy = (min(xvalues) - diff_x/2, min(yvalues))
        rect = Rectangle(xy, width, height, facecolor='white', alpha=alpha)
        ax.add_patch(rect)
        #if not no_deco:
        #    ax.text(xy[0] + width/2, xy[1]+height/2, "used for\ncalibration", fontdict={'color':'black', 'ha':'center', 'va':'bottom'})
    return fig, ax

In [ ]:
def plot_positions(ax, positions_cm, walls):
    from matplotlib import cm
    
    cmap = cm.get_cmap('inferno') 
    n_labels = 3
    label = None
    step = len(positions_cm) // n_labels
    for i, p in enumerate(positions_cm):
        if i % step == 0 or (i == len(positions_cm) - 1):
            label = f'position {i}'
        ax.scatter(*p[:2], color=cmap(i / len(positions_cm)), label=label)
        label=None
    ax.plot(positions_cm[:, 0], positions_cm[:, 1], color='k', ls=':')
    ax.axis('equal')
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('y [cm]')
    handles, labels = ax.get_legend_handles_labels()
    ax.legend([handles[0], handles[-1]], ["start", "end"], loc='upper right')
    
    for wall_distance, wall_angle in walls:
        print(wall_distance, wall_angle)
        ax.arrow(0, 0, wall_distance * np.cos(wall_angle / 180 * np.pi), 
                 wall_distance * np.sin(wall_angle / 180 * np.pi), color='k', label='_wall', 
                 width=1)
    ax.grid()

## 1. Single wall approach experiments

### 1.1 Qualitative evaluation

In [ ]:
from crazyflie_description_py.experiments import WALL_ANGLE_DEG
from crazyflie_description_py.parameters import FLYING_HEIGHT_CM
from generate_classifier_results import get_groundtruth_distances, WALLS_DICT

from utils.constants import SPEED_OF_SOUND
from utils.plotting_tools import FIGSIZE, save_fig

exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; 
#appendix="_new3" # best
#appendix="_new6" # too fast, otherwise good
#appendix="_new10" # okay-ish
#appendix="_new12"
#appendix="_new4" # frequencies wrong
#appendix="_new7" # frequencies wrong
#appendix="_new8" # frequencies wrong
appendix="_1" # very good
#appendix="_2" # not good
#appendix="_3" # okay
#appendix="_4" # good
#appendix="_5"A # not good
#appendix="_6"

#exp_name="2022_01_27_demo"; motors="live"; 
#appendix = "test4"

#exp_name="2021_07_27_epuck_wall"; motors="sweep_and_move"; appendix = ""
#exp_name="2021_07_08_stepper_fast"; motors="all45000"; appendix = ""

results_df = pd.read_pickle(f"../datasets/{exp_name}/all_data.pkl")
row = results_df.loc[
    (results_df.appendix == appendix) & (results_df.motors == motors), :
].iloc[0]

bad_freqs = []
n_calib = 5

flying = row.positions[:, 2] > FLYING_HEIGHT_CM * 1e-2
positions_cm = row.positions[flying, :2] * 1e2
yaws_deg = row.positions[flying, 3]
freqs = row.frequencies_matrix[0, :]
magnitudes = np.abs(row.stft[flying][:, :, freqs>0])
freqs = freqs[freqs > 0]
n_points = magnitudes.shape[0]
times = row.seconds[flying]
distances = get_groundtruth_distances(exp_name, appendix, flying=True)

from utils.constants import SPEED_OF_SOUND
delta_f = np.min(np.diff(freqs))
print("minimum difference", delta_f)
max_d = SPEED_OF_SOUND / (4 * delta_f)
print("maximum distance:", max_d)

# calibrate and plot calibration
calib_on_the_fly = np.median(magnitudes[:n_calib, :, :], axis=0)
std_values = np.std(magnitudes[:n_calib, :, :], axis=0)
plot_name_calib = f"plots/experiments/{exp_name}{appendix}_on_the_fly.pdf"
fig_calib, axs_calib = plt.subplots(1, 5, sharey=True)
fig_calib.set_size_inches(10, 3)
axs_calib[0].set_ylabel(f"amplitude [-]")
for m in range(4):
    axs_calib[m].plot(freqs, magnitudes[:n_calib, m, :].T, color=f"C{m}", marker='o')
    axs_calib[m].set_title(f"mic{m}")
    axs_calib[4].errorbar(freqs, calib_on_the_fly[m], std_values[m], label=f"mic{m}")
    axs_calib[m].set_xlabel(f"frequency [Hz]")
axs_calib[4].set_xlabel(f"frequency [Hz]")
axs_calib[4].set_title(f"all mics")
axs_calib[0].set_ylim(2, 12)
[axs_calib[0].axvline(f, color='black') for fs in bad_freqs for f in fs]

# plot positions
fig, ax = plt.subplots()
fig.set_size_inches(3, 3)
plot_positions(ax, positions_cm, WALLS_DICT[exp_name])
plot_name = f"plots/experiments/{exp_name}{appendix}_positions.pdf"

### 1.2 Quantitative evaluation

In [ ]:
mics_beam = np.array([[-0.011,  0.048,  0.019,  0.019], [-0.018, -0.017,  0.014, -0.048]])
room_mics =  np.array([[9.418, 9.417, 9.386, 9.448], [3.489, 3.548, 3.519, 3.519]])

plt.figure()
for i, m in enumerate(mics_beam.T):
    plt.scatter(*m, label=f"mic{i}")
plt.axis("equal")
plt.legend()

plt.figure()
#for i, m in enumerate(mics_beam.T):
for i, m in enumerate(room_mics.T):
    plt.scatter(*m, label=f"mic{i}")
plt.axis("equal")
plt.legend()

In [ ]:
from generate_classifier_results import get_groundtruth_distances
from plot_helpers import ls, labels, add_double_legend
from generate_filtering_results import angle_error
np.random.seed(2)

D_RANGE = [7, 80]
PLOT_D_MAX = 30

no_deco = True
fig_size = (8, 4)

chosen_calibration_param = 0.3
simplify_angles = False
fig_size_cdf = (10, 4)

exp_name="2021_10_12_flying"
name = "flying"; appendices = ["_new3", "_new6"] + [f"_{i}" for i in range(1, 7)] 

#exp_name="2021_07_08_stepper_fast"
#name = "stepper"; appendices=[""]

#exp_name="2021_07_27_epuck_wall"
#name = "epuck"; appendices=[""]

#fig_cdf.suptitle(f"angles simplified {simplify_angles}")

beamform = False
discretization = "fine"

color = {f"exp{a}": "C0" if "new" in a else "C1" for i, a in enumerate(appendices) }
color["random"] = "k"
color["fixed"] = "k"

#for estimator in ["moving"]: #["particle"]:#, "moving"]:
for estimator in ["particle"]:
#for estimator in ["histogram", "moving", "particle"]:#, "moving"]:
    fig_cdf, axs_cdf = plt.subplots(1, 2)
    fig_cdf.set_size_inches(*fig_size_cdf)
    for i_app, appendix in enumerate(appendices):
        
        # flow deck was drifting a lot for experiments downstairs (carpet)
        # so we use the wall as a reference for ground truth distance (the drone always hit the wall)
        if not "_new" in appendix:
            correct = True
        else:
            correct = False
            
        gt_distances, gt_angles = get_groundtruth_distances(exp_name, appendix, 
                                                            flying=True, angles=True, correct=correct)
        
        if beamform:
            fname = f"results/{name}_results_matrices_{estimator}{appendix}_beamform_new.pkl"
        else:
            fname = f"results/{name}_results_matrices_{estimator}{appendix}_new.pkl"
        df_mat = pd.read_pickle(fname) 
        print("read", fname)
        df_here = df_mat[
            (df_mat["calibration param"]==chosen_calibration_param) & 
            (df_mat["simplify angles"]==simplify_angles) 
        ]
        for i, row in df_here.iterrows():
            if estimator == "particle":
                method = "particle"
            elif estimator == "moving":
                method = f"histogram {row['n window']}"
            else:
                method = "histogram"
                
            if appendix == appendices[0]:
                label = labels[method]
                #print("label", label)
            else:
                label = None
                
            n_calib_here = int(np.floor(row["calibration param"]))
            d_here = row["distances_cm"]
            a_here = row["angles_deg"]
            matrix_distances = row["matrix distances"]
            matrix_angles = row["matrix angles"]

            d_estimates = get_estimates_here(matrix_distances, d_here)

            cdf_distances = np.sort(np.abs(d_estimates[n_calib:] - gt_distances[n_calib:]))
            y = np.linspace(0, 1, len(cdf_distances))
            axs_cdf[0].plot(cdf_distances, y, ls=ls[method], label=label, color=color[f"exp{appendix}"])

            if not row["simplify angles"]:
                a_estimates = get_estimates_here(matrix_angles, a_here)
                
                error = angle_error(a_estimates, gt_angles)
                cdf_angles = np.sort(np.abs(error))
                y = np.linspace(0, 1, len(cdf_angles))
                axs_cdf[1].plot(cdf_angles, y, label=label, ls=ls[method], color=color[f"exp{appendix}"])
                
            fig, ax = plot_matrix(d_here, matrix_distances, #xvalues=times, 
                                  gt_values=gt_distances,
                                  no_deco=no_deco, n_calib=n_calib_here)
            ax.set_title(f" exp{appendix} {estimator}".replace("_", "\\_"))
            fig.set_size_inches(*fig_size)
            
            fig, ax = plot_matrix(a_here, matrix_angles, 
                                  gt_values=gt_angles, 
                                  no_deco=no_deco, n_calib=n_calib_here)
            ax.set_title(f" exp{appendix} {estimator}".replace("_", "\\_"))
            fig.set_size_inches(*fig_size)

    # generate random
    d_random = np.random.choice(range(*D_RANGE), size=len(gt_distances))
    a_random = np.random.choice(range(360), size=len(gt_angles))
    lines = []; leg_labels = []
    axs_cdf[0].plot(sorted(np.abs(d_random-gt_distances)), np.linspace(0, 1, len(gt_distances)), color=color["random"])
    line, = axs_cdf[1].plot(sorted(np.abs(angle_error(a_random, gt_angles))), np.linspace(0, 1, len(gt_angles)), color=color["random"])
    lines.append(line); leg_labels.append("random")
    
    # generate fixed
    d_fixed = (D_RANGE[1]-D_RANGE[0]) / 2
    a_fixed = gt_angles[0] + 180 # 
    line, = axs_cdf[0].plot(sorted(np.abs(d_fixed-gt_distances)), np.linspace(0, 1, len(gt_distances)), ls=":", color=color["fixed"])
    lines.append(line); leg_labels.append("fixed")

    axs_cdf[1].plot(sorted(np.abs(angle_error(a_fixed, gt_angles))), np.linspace(0, 1, len(gt_angles)), ls=":", color=color["fixed"])
    line, = axs_cdf[1].plot([], [], color="C0")
    lines.append(line); leg_labels.append("dataset glass wall")
    line, = axs_cdf[1].plot([], [], color="C1")
    lines.append(line); leg_labels.append("dataset whiteboard")
    axs_cdf[1].legend(lines, leg_labels, title="used data", loc="upper left", bbox_to_anchor=[1.0, 0.5])

    axs_cdf[0].grid(True)
    axs_cdf[1].grid(True)
    axs_cdf[0].set_xlabel("absolute error [cm]")
    axs_cdf[1].set_xlabel("absolute error [deg]")
    axs_cdf[0].set_title("distance estimation")
    axs_cdf[1].set_title("angle estimation")
    axs_cdf[0].set_ylabel("cdf [-]")
    axs_cdf[1].set_ylabel("cdf [-]")
    axs_cdf[0].set_xlim(0, PLOT_D_MAX)
    axs_cdf[1].set_xlim(0, 200)
    fig_cdf.set_size_inches(*fig_size)
    fig_cdf.subplots_adjust(wspace=0.3)
    if beamform:
        save_fig(fig_cdf, f"plots/experiments/{exp_name}_{discretization.replace(' ','_')}_{estimator}_beamform_cdfs.pdf")
    else:
        save_fig(fig_cdf, f"plots/experiments/{exp_name}_{discretization.replace(' ','_')}_{estimator}_cdfs.pdf")

In [ ]:
### Comparison of datasets

In [ ]:
no_deco=False

color = {"epuck": "C0", "stepper": "C1"}
color["random"] = "k"
color["fixed"] = "k"

estimator = "particle"
appendix = ""

fig_cdf, axs_cdf = plt.subplots(1, 2, sharey=True)
fig_cdf.set_size_inches(*fig_size_cdf)

linestyles = {
    "pyroom": ":",
    "": "-"
}
names = {
    "pyroom": " simulated",
    "": ""
}

fig_hor, (ax_dist, ax_ang) = plt.subplots(2, 1, sharex=True)

for exp_name in ["2021_07_08_stepper_fast", "2021_07_27_epuck_wall"]:
    if exp_name == "2021_07_27_epuck_wall":
        print("epuck")
        from epuck_description_py.experiments import DISTANCES_CM, WALL_ANGLE_DEG_STEPPER
        gt_distances = DISTANCES_CM
        gt_angles = np.full(len(DISTANCES_CM), WALL_ANGLE_DEG_STEPPER)
        name = "epuck"
    elif exp_name == "2021_07_08_stepper_fast":
        print("stepper")
        from crazyflie_description_py.experiments import DISTANCES_CM, WALL_ANGLE_DEG_STEPPER
        gt_distances = DISTANCES_CM
        gt_angles = np.full(len(DISTANCES_CM), WALL_ANGLE_DEG_STEPPER)
        name = "stepper"
        
    for simulate in ["", "pyroom"]:
        fname = f"results/{name}_results_matrices_{estimator}{appendix}{simulate}_new.pkl"
        df_here = pd.read_pickle(fname) 
        for i, row in df_here.iterrows():
            d_here = row["distances_cm"]
            a_here = row["angles_deg"]
            matrix_distances = row["matrix distances"]
            matrix_angles = row["matrix angles"]

            d_estimates = get_estimates_here(matrix_distances, d_here)
            gt_distances_here = row.gt_distances_cm
            gt_angles_here = row.gt_angles_deg

            d_errors = d_estimates - gt_distances_here
            cdf_distances = np.sort(np.abs(d_errors))
            y = np.linspace(0, 1, len(cdf_distances))
            axs_cdf[0].plot(cdf_distances, y, ls=linestyles[simulate], label=name+names[simulate], color=color[name])
            ax_dist.plot(gt_distances_here, d_errors, color=color[name], ls=linestyles[simulate])

            a_estimates = get_estimates_here(matrix_angles, a_here)

            a_errors = angle_error(a_estimates, gt_angles_here)
            
            ax_ang.plot(gt_distances_here, a_errors, color=color[name], ls=linestyles[simulate])
            
            cdf_angles = np.sort(np.abs(a_errors))
            y = np.linspace(0, 1, len(cdf_angles))
            axs_cdf[1].plot(cdf_angles, y, ls=linestyles[simulate], label=name+names[simulate], color=color[name])

            fig, ax = plot_matrix(d_here, matrix_distances, #xvalues=times, 
                                  gt_values=gt_distances_here,
                                  no_deco=no_deco)
            ax.set_title(name)
            fig.set_size_inches(*fig_size)

            fig, ax = plot_matrix(a_here, matrix_angles, 
                                  gt_values=gt_angles_here, 
                                  no_deco=no_deco)
            ax.set_title(name)
            fig.set_size_inches(*fig_size)

# generate random
d_random = np.random.choice(range(7, 81), size=len(gt_distances_here))
a_random = np.random.choice(range(360), size=len(gt_angles_here))
axs_cdf[0].plot(sorted(np.abs(d_random-gt_distances_here)), np.linspace(0, 1, len(gt_distances_here)), color=color["random"], label="random")
axs_cdf[1].plot(sorted(np.abs(angle_error(a_random, gt_angles_here))), np.linspace(0, 1, len(gt_angles_here)), color=color["random"], label="random")

# generate fixed
d_fixed = (50-7)/2
a_fixed = gt_angles[0] + 180
axs_cdf[0].plot(sorted(np.abs(d_fixed-gt_distances_here)), np.linspace(0, 1, len(gt_distances_here)), ls=":", color=color["fixed"], label="fixed")
axs_cdf[1].plot(sorted(np.abs(angle_error(a_fixed, gt_angles_here))), np.linspace(0, 1, len(gt_angles_here)), ls=":", color=color["fixed"], label="fixed")
axs_cdf[1].legend(title="used data", loc="upper left", bbox_to_anchor=[1.0, 0.75])

axs_cdf[0].grid(True)
axs_cdf[1].grid(True)
axs_cdf[0].set_xlabel("absolute distance error [cm]")
axs_cdf[1].set_xlabel("absolute angle error [deg]")
axs_cdf[0].set_title("distance estimation")
axs_cdf[1].set_title("angle estimation")
axs_cdf[0].set_ylabel("cdf [-]")
#axs_cdf[1].set_ylabel("cdf [-]")
axs_cdf[0].set_xlim(0, PLOT_D_MAX)
axs_cdf[1].set_xlim(0, 200)
fig_cdf.set_size_inches(8, 4)
#fig_cdf.subplots_adjust(wspace=0.3)

fig_hor.set_size_inches(4, 4)
ax_ang.set_xlabel("distance [cm]")
#ax_dist.set_xlabel("distance [cm]")
ax_dist.set_title("error evolution")
ax_ang.set_ylabel("angle error [deg]", x=1.5)
ax_ang.grid()
ax_dist.grid()
ax_dist.set_ylabel("distance error [cm]")
if beamform:
    save_fig(fig_cdf, f"plots/experiments/comparison_{estimator}_beamform_cdfs.pdf")
    save_fig(fig_hor, f"plots/experiments/comparison_{estimator}_beamform.pdf")
else:
    save_fig(fig_cdf, f"plots/experiments/comparison_{estimator}_cdfs.pdf")
    save_fig(fig_hor, f"plots/experiments/comparison_{estimator}.pdf")

### 1.3 Factor graph inference

In [ ]:
from factor_graph.plot import plot_projections
import gtsam
from audio_gtsam.wall_backend import WallBackend, add_decorations

X = gtsam.symbol_shorthand.X
P = gtsam.symbol_shorthand.P

wall_backend = WallBackend(use_isam=True)
yaw_start = np.pi 

use_groundtruth = False

dist_matrix = results_matrix_moving
angle_matrix = results_matrix_angles_moving
n_estimates = 1

d_estimates = []
d_gt = []

fig, ax = plt.subplots()

for i, prob_dists in enumerate(dist_matrix.T):
    prob_dists = dist_matrix[:, i]
    t1 = time.time()
    if i < n_calib:
        continue
    
    if len(positions_cm[i, :]) < 3:
        position_cm = np.r_[positions_cm[i, :], 0.0]
    else:
        position_cm = positions_cm[i, :]
    yaw = yaws_deg[i] / 180 * np.pi
    pose_factor = wall_backend.add_pose(r_world=position_cm * 1e-2,  yaw=yaw, verbose=False)
    
    distance_gt = distances[i]
    
    if use_groundtruth:
        prob_dists = np.zeros(len(distances_cm))
        idx = np.where(distances_cm == distance_gt)[0]
        prob_dists[idx] = 1.0
        
        prob_angles = np.zeros(len(angles_deg))
        prob_angles[angles_deg==90] = 1.0
    else:
        prob_angles = angle_matrix[:, i]
        
    wall_backend.add_planes_from_distributions(np.array(distances_cm), prob_dists, 
                                               np.array(angles_deg), prob_angles, 
                                               limit_distance=20,
                                               n_estimates=n_estimates, 
                                               verbose=False)
    
    wall_backend.check_wall(verbose=False)
    distance_estimate = wall_backend.get_distance_estimate()
    d_estimates.append(distance_estimate * 1e2 if distance_estimate else np.nan)
    d_gt.append(distance_gt if distance_estimate else np.nan)
    
    wall_backend.plot(fig, ax, n_poses=20)
add_decorations(fig, ax, n_poses=20)

errors_df.loc[len(errors_df), :] = {
    "algorithm": algorithm + f"_win{n_window}_factorgraph",
    "mics": mics_str,
    "calib_method": calib_method,
    "estimates": d_estimates,
    "distances": d_gt,
    "appendix": appendix,
    "time": time_moving / n_points 
}

## 2. Multi-wall approach experiments 

In [ ]:
exp_name = "2021_11_23_demo"
results_df = pd.read_pickle(f"../datasets/{exp_name}/all_data.pkl")
print("available appendices:", results_df.appendix.values)

# glass wall
#appendix = "hover1" # works super well! 
#appendix = "hover3" # looks good! 
#appendix = "hover5" # looks quite good! 
#appendix = "hover6" # looks ok (from here on, velocity is increased)
appendix = "hover8" # works well (glass wall), but only detects one wall

# normal wall
#appendix = "hover9" # works not great
#appendix = "hover10" # works not great
#appendix = "hover11" # works okay
#appendix = "hover12" # works okay 

In [ ]:
exp_name = "2022_01_27_demo"
results_df = pd.read_pickle(f"../datasets/{exp_name}/all_data.pkl")
print("available appendices:", results_df.appendix.values)

appendix = "test4"  # works well
#appendix = "test3" # works ok

In [ ]:
from utils.plotting_tools import FIGSIZE, save_fig, add_colorbar
from crazyflie_demo.wall_detection import FLYING_HEIGHT_CM
from generate_classifier_results import WALLS_DICT, get_groundtruth_distances

row = results_df.loc[results_df.appendix == appendix,:].iloc[0]

positions_cm = row.positions[:, :3] * 1e2
flying = (positions_cm[:, 2] > FLYING_HEIGHT_CM)  & (positions_cm[:, 2] < 100) & (np.abs(positions_cm[:, 1]) < 100) & (np.abs(positions_cm[:, 0]) < 50)
positions_cm = positions_cm[flying, :]
yaws_deg = row.positions[flying, 3]
freqs = row.frequencies_matrix[0, :]
magnitudes = np.abs(row.stft[flying][:, :, freqs>0])
signals_f = row.stft[flying][:, :, freqs>0]
freqs = freqs[freqs > 0]
times = row.seconds[flying]

n_timesteps = magnitudes.shape[0] 

distances_walls = get_groundtruth_distances(exp_name=exp_name, appendix=appendix)

fig, ax = plt.subplots()
fig.set_size_inches(FIGSIZE, FIGSIZE)
plot_positions(ax, positions_cm[:n_timesteps, :], walls=WALLS_DICT[exp_name])
plot_name = f"plots/experiments/{exp_name}{appendix}_positions.pdf"

In [ ]:
def plot_groundtruth(ax, positions_cm, walls, n_poses=20):
    from audio_gtsam.wall_backend import plot_wall, add_decorations
    cmap = plt.get_cmap('inferno', n_timesteps) 
    n_labels = 3
    
    for i, p in enumerate(positions_cm):
        ax.scatter(*p[:2]*1e-2, color=cmap(i))
        
    for i, (wall_distance, wall_angle) in enumerate(walls):
        normal = np.r_[
            np.cos(wall_angle / 180 * np.pi), 
            np.sin(wall_angle / 180 * np.pi)
        ]
        arrow, line = plot_wall(wall_distance*1e-2, normal, ax, plane_index=i, label=f"real $\\pi^{{({i})}}$", ls=":", lw=3.0)
    add_decorations(fig, ax, n_poses=n_poses)

In [ ]:
from audio_gtsam.wall_backend import WallBackend
from audio_gtsam.wall_backend import add_decorations

from utils.moving_estimators import get_estimate
from crazyflie_demo.wall_detection import WallDetection
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

import time

estimation_method = "mean"
estimator = "moving"

no_deco = False
WallDetection.SIMPLIFY_ANGLES = True
wall_detection = WallDetection(python_only=True, estimator=estimator)
print(WallDetection.SIMPLIFY_ANGLES)
wall_backend = WallBackend(use_isam=True)

fg_dist_estimates = []
fg_colors = []
fg_angle_estimates = []

angles_forward = []

distance_estimates = []
std_estimates = []

results_matrix_angles = np.full((len(wall_detection.estimator.angles_deg), n_timesteps), np.nan)
results_matrix_moving = np.full((len(wall_detection.estimator.distances_cm), n_timesteps), np.nan)

fig, ax = plt.subplots()
plot_groundtruth(ax, positions_cm, walls=WALLS_DICT[exp_name], n_poses=0)

runtimes_fg = []

for i in range(n_timesteps):
    if i % 20 == 0:
        print(f"{i+1}/{n_timesteps}")
    res = wall_detection.listener_callback_offline(
        signals_f[i].T, freqs, positions_cm[i], yaws_deg[i], timestamp=times[i]*1e3
    )
    #fig, ax = plt.subplots()
    #ax.pcolorfast(wall_detection.calibration_data[0, :, :])
    
    if i <= WallDetection.N_CALIBRATION:
        fg_dist_estimates.append(None)
        fg_angle_estimates.append(None)
        fg_colors.append("k")
        
        angles_forward.append(None)
        
        distance_estimates.append(None)
        std_estimates.append(None)
        continue
        
    if res is None:
        print("no result yet!")
        
    #plt.plot(freqs, wall_detection.calibration[0, :])
    #plt.plot(freqs, wall_detection.calibration_std[0, :])
        
    #angle_local = wall_detection.estimator.get_local_forward_angle()
    #angles_forward.append(angle_local)
        
    __, __, prob_moving_dist, prob_moving_angle = res
        
    results_matrix_moving[:, i] = prob_moving_dist
    
    t1 = time.time()
    d, std = get_estimate(wall_detection.estimator.distances_cm, prob_moving_dist, method=estimation_method)
    distance_estimates.append(d)
    std_estimates.append(std)
    
    if not WallDetection.SIMPLIFY_ANGLES:
        results_matrix_angles[:, i] = prob_moving_angle
    
    yaw = yaws_deg[i] / 180 * np.pi
    pose_factor = wall_backend.add_pose(r_world=positions_cm[i] * 1e-2,  yaw=yaw, verbose=False)
    #wall_backend.add_planes_from_distributions(distances_cm, prob_moving_dist, angles_deg, prob_moving_angle, n_estimates=1, verbose=True)
    
    wall_backend.add_plane_from_distances(wall_detection.estimator.distances_cm, prob_moving_dist, verbose=False, method=estimation_method)
    #wall_backend.check_wall(verbose=True)
    
    d_estimate = wall_backend.get_distance_estimate()
    angle_deg = wall_backend.get_angle_estimate()
    runtimes_fg.append(time.time() - t1)
    
    fg_dist_estimates.append(d_estimate * 1e2 if d_estimate else None)
    fg_angle_estimates.append(angle_deg)
    fg_colors.append(f"C{wall_backend.plane_index+1}")
    
    wall_backend.plot(fig, ax, n_poses=n_timesteps, live_update=False)
wall_backend.plot(fig, ax, n_poses=n_timesteps, live_update=False, final=True)
#add_decorations(fig, ax, n_poses=n_timesteps)
add_decorations(fig, ax, n_poses=0)
save_fig(fig, f"plots/experiments/{exp_name}{appendix}_result.pdf")
#plt.plot(times[:n_timesteps], angles_forward)
print("done")

In [ ]:
plt.figure()
plt.title("factor graph runtime")
plt.plot(np.array(runtimes_fg) * 1e3)
plt.axhline(np.mean(runtimes_fg) * 1e3)
plt.xlabel("time index [-]")
plt.ylabel("time [ms]")
plt.figure()
plt.title("inference runtime")
plt.plot(np.array(runtimes_inference) * 1e3)
plt.axhline(np.mean(runtimes_inference) * 1e3)
plt.xlabel("time index [-]")
plt.ylabel("time [ms]")

# Sandbox

In [ ]:
from generate_classifier_results import DIST_THRESH, STD_THRESH, TAIL_THRESH

def plot_distance_matrix(ax, matrix, distances_cm=None):
    from utils.plotting_tools import pcolorfast_custom
    from copy import deepcopy
    
    if distances_cm is None:
        distances_cm = wall_detection.estimator.distances_cm

    cmap = plt.get_cmap("gray")
    alpha = 0.8
    cmap.set_bad((1 - alpha, 1 - alpha, 1 - alpha))
    log_matrix = deepcopy(matrix)
    log_matrix[matrix > 0] = np.log10(matrix[matrix > 0])
    log_matrix[matrix <= 0] = np.nan
    pcolorfast_custom(
        ax,
        np.arange(log_matrix.shape[1]),
        distances_cm,
        log_matrix,
        n_xticks=12,
        n_yticks=4,
        cmap=cmap,
        alpha=alpha,
    )
    ax.set_ylabel("distance [cm]")


def plot_groundtruth_d(ax, color="white", **kwargs):
    ax.plot(distances_walls, color=color, **kwargs)

def plot_distance_estimates(ax, distance_estimates, label=None, **kwargs):
    ax.plot(distance_estimates, label=label, **kwargs)
    ax.set_ylabel("distance [cm]")
    ax.axhline(DIST_THRESH, color="k", ls="--")


def plot_std_estimates(ax, std_estimates, label=None, **kwargs):
    ax.plot(std_estimates, label=label, **kwargs)
    ax.set_ylabel("standard deviation [cm]")
    ax.axhline(STD_THRESH, color="k", ls="--")
    ax.set_yscale("log")
    ax.grid(True, which="both")


def plot_tail(ax, matrix, **kwargs):
    tail = np.log10(np.mean(matrix[-3:, :], axis=0))
    ax.plot(tail, **kwargs)
    ax.set_ylabel("tail probability")
    ax.grid(True)
    ax.axhline(TAIL_THRESH, color="k", ls="--")


def plot_angles(ax, matrix, **kwargs):
    matrix_norm = matrix / np.sum(matrix, axis=0)[None, :]
    for i in range(matrix_norm.shape[0]):
        angle = ANGLES_DEG[i]
        ax.plot(matrix_norm[i, :], label=f"wall at {angle}", **kwargs)
    ax.legend(loc="upper right")
    ax.grid(True)
    ax.set_ylabel("angle probability")
    
def plot_fg_estimates(ax, d_estimates, colors):
    ax.scatter(np.arange(len(d_estimates)), d_estimates, color=colors)

def plot_all():
    fig, axs = plt.subplots(4, 1, sharex=True)
    fig.set_size_inches(20, 30)

    plot_distance_matrix(axs[0], results_matrix_moving)
    ymin, ymax = axs[0].get_ylim()
    plot_groundtruth_d(axs[0], color="white")
    plot_distance_estimates(axs[0], distance_estimates, label="chosen method")
    
    plot_fg_estimates(axs[0], fg_dist_estimates, fg_colors)
    axs[0].set_ylim(ymin, ymax)
    
    plot_groundtruth_d(axs[1], color="black")
    plot_distance_estimates(axs[1], distance_estimates, label="chosen method")
    plot_fg_estimates(axs[1], fg_dist_estimates, fg_colors)
    axs[1].set_ylim(5, 100)
    axs[1].grid(True)

    plot_std_estimates(axs[2], std_estimates, label="chosen method")

    #plot_tail(axs[3], results_matrix_moving)
    if not WallDetection.SIMPLIFY_ANGLES:
        plot_angles(axs[3], results_matrix_angles)
    return fig, axs

In [ ]:
estimator = "particle"
appendix = "test3"
fname = f"results/demo_results_matrices_{estimator}{appendix}.pkl"
try:
    matrix_df = pd.read_pickle(fname)
except FileNotFoundError:
    print("Could not find", fname)
    print("Run generate_flying_results.py to generate results.")
matrix_df

In [ ]:
methods = ["mean", "peak", "max"]
row = matrix_df.iloc[0]
matrix = row['matrix distances'][0]

fig, axs = plt.subplots(4, 1, sharex=True)
fig.set_size_inches(20, 20)

plot_groundtruth_d(axs[0], color="white")
plot_groundtruth_d(axs[1], color="black")
plot_tail(axs[3], matrix)

# try different wall detection schemes
for method in methods:
    distance_estimates = []
    std_estimates = []
    
    for i, prob_moving_dist in enumerate(matrix.T):
        
        d, std = get_estimate(row.distances_cm, prob_moving_dist, method=method)
        distance_estimates.append(d)
        std_estimates.append(std)
    print(len(distance_estimates))
        
    plot_distance_estimates(axs[0], distance_estimates, label=method)
    plot_distance_estimates(axs[1], distance_estimates, label=method)
    plot_std_estimates(axs[2], std_estimates, label=method)
    
axs[1].grid(True)
axs[1].set_ylim(5, 100)

#if not WallDetection.SIMPLIFY_ANGLES:
#    plot_angles(axs[4], results_matrix_angles)
plot_distance_matrix(axs[0], matrix, row.distances_cm)

axs[0].legend(loc='upper right')
axs[1].legend(loc='upper right')
#axs[0].set_ylim(ymin, ymax)

axs[0].set_title(get_title(row))

In [ ]:
# based on above, choose parameters. 
from generate_classifier_results import get_groundtruth_distances, get_precision_recall, THRESHOLDS_DICT
from utils.pandas_utils import filter_by_dict

fig, ax = plt.subplots()
fig.set_size_inches(4, 4)
delta = 0.05
    
for estimator in ["particle", "moving"]:
    try:
        matrix_df = pd.read_pickle(f"results/demo_results_matrices_{estimator}{appendix}.pkl")
    except FileNotFoundError:
        print("Run generate_flying_results.py to generate results.")

    n_windows = matrix_df["n window"].unique()
    print(n_windows)
    chosen_method = "distance-mean"
    chosen_dict = {
        "n window": None, # is filled below
        "simplify angles": False,
        "relative std": 0.0,
        "calibration name": "iir",
        "calibration param": 0.3,
        "mask bad": "fixed"
    }

    distances_wall = get_groundtruth_distances("2022_01_27_demo", "test4") 
    matrix_df.loc[~matrix_df["mask bad"].isin(["adaptive", "fixed"]), "mask bad"] = "None"
    matrix_df = matrix_df.apply(pd.to_numeric, errors='ignore', axis=0)
    matrix_df["calibration param"] = np.round(matrix_df["calibration param"], 1)

    for i_window, n_window in enumerate(n_windows):
        chosen_dict["n window"] = n_window
        rows = filter_by_dict(matrix_df, chosen_dict)
        #[print(key, matrix_df[key].unique()) for key in chosen_dict.keys()]
        if len(rows) != 1:
            print("error:", matrix_df, chosen_dict)
            continue
        matrix = rows.iloc[0]["matrix distances"][0]
        precision, recall = get_precision_recall(matrix, distances_wall, method=chosen_method, verbose=False, sort=False) 

        if estimator == "moving":
            label = f"histogram $N_w={n_window}$"
        else:
            label = "particle filter"
        ax.plot(precision, recall, marker='o', label=label)

        thresholds = THRESHOLDS_DICT[chosen_method]
        for t, p, r in zip(thresholds, precision, recall):
            if False: #(n_window == 5) and (r < 1):
                ax.annotate(t, (p + -delta, r+delta))

        #fig_mat, ax_mat = plt.subplots()
        #ax_mat.pcolorfast(np.arange(matrix.shape[1]), DISTANCES_CM, np.log10(matrix))
        #ax_mat.plot(DISTANCES_CM[np.argmax(matrix, axis=0)], color='white')
    ax.set_xlabel("precision")
    ax.set_ylabel("recall")
    ax.set_xlim(-0.1, 1.1)
    ax.set_ylim(-0.1, 1.1)
    ax.grid(True)
    plt.tight_layout()
    ax.legend(loc='lower left') #bbox_to_anchor=[1.0, 1.0])
save_fig(fig, f"plots/experiments/{exp_name}{appendix}_pr_curves.pdf")

In [ ]:
try:
    scores = pd.read_pickle(f"results/demo_results_classifier_{estimator}.pkl")
    scores.loc[~scores["mask bad"].isin(["adaptive", "fixed"]), "mask bad"] = "None"
except FileNotFoundError:
    print("File not found. Generate it using generate_classifier_results.py")

In [ ]:
def plot_grid(
    df_here, 
    x_name = "calibration param",
    y_name = "auc",
    row_name = None,
    col_name = None,
    color_name = None,
    marker_name = None,
    linestyle_name = None,
):
    
    from matplotlib.lines import Line2D
    marker_list = list(Line2D.markers.keys())[1:]
    linestyle_list = list(Line2D.lineStyles.keys())
    
    groupby = {}

    if color_name is not None:
        colors = {col: f"C{i}" for i, col in enumerate(df_here[color_name].unique())}
        groupby["color"] = color_name
    else:
        colors = ["C0"]
    
    if marker_name is not None:
        markers = {mark: marker_list[i] for i, mark in enumerate(df_here[marker_name].unique())}
        groupby["marker"] = marker_name
    else:
        markers = [""]
    
    if linestyle_name is not None:
        linestyles = {ls: linestyle_list[i] for i,ls in enumerate(df_here[linestyle_name].unique())}
        groupby["ls"] = linestyle_name
    else:
        linestyles = ["-"]
    
    rows = df_here[row_name].unique()
    cols = df_here[col_name].unique()
    fig, axs = plt.subplots(len(rows), len(cols), sharey=True, squeeze=False)
    fig.set_size_inches(5 * len(cols), 5 * len(rows))
    for i, row in enumerate(rows):
        for j, col in enumerate(cols):
            df_plot = df_here.loc[(df_here[row_name] == row) & (df_here[col_name] == col)]
            
            for groupby_vals, df_col in df_plot.groupby(list(groupby.values())):
                
                groupby_dict = dict(zip(groupby.keys(), groupby_vals))
                
                # if these elements are not tin the keys, then we access
                # 0th element of corresponding list.
                l = groupby_dict.get("ls", 0)
                c = groupby_dict.get("color", 0)
                m = groupby_dict.get("marker", 0)
                
                axs[i, j].plot(df_col[x_name], df_col[y_name], ls=linestyles[l], marker=markers[m], color=colors[c])
            axs[i, j].grid(True)
    [axs[0, j].set_title(f"{col_name}:\n{col}") for j, col in enumerate(cols)]
    [axs[i, -1].twinx().set_ylabel(f"{row_name}:\n{row}") for i, row in enumerate(rows)]
    [axs[i, -1].twinx().set_yticks([]) for i, row in enumerate(rows)]
    
    if linestyle_name is not None:
        for label, ls in linestyles.items():
            axs[0, -1].plot([], [], ls=ls, label=label, color="C0")
        axs[0, -1].legend(title=linestyle_name)    
    
    if marker_name is not None:
        for label, marker in markers.items():
            axs[-1, -1].plot([], [], marker=marker, label=label, color="C0")
        axs[-1, -1].legend(title=marker_name)    
        
    if color_name is not None:
        for label, color in colors.items():
            axs[-1, -1].plot([], [], color=color, label=label)
        axs[-1, -1].legend(title=color_name)    
    return fig, axs

In [ ]:
df_here = scores[scores.method == "distance-mean"]
fig, axs = plot_grid(df_here, 
    row_name = "mask bad",
    col_name = "calibration name",
    x_name = "calibration param",
    y_name = "auc",
    color_name = "n window",
    marker_name = "simplify angles",
    linestyle_name = "relative std"
)

In [ ]:
### alternative angle calculation: geometric averaging based on mic level differences

mic_angles = np.array([90, 180, -90, 0])
mics = [0, 1, 2, 3]

total = np.sum(magnitudes[:, :, :]**2, axis=(0, 2))
powers = np.sum(magnitudes[:, mics]**2, axis=2) / total[None, :]

fig = plt.figure()
for i, mic in enumerate(mics): #range(powers.shape[1]):
    plt.plot(range(powers.shape[0]), powers[:, i], label=f"mic{mic}")
plt.legend()
fig.set_size_inches(10, 5)

fig = plt.figure()
norm = powers[:] / np.mean(powers, axis=1)[:, None]
print(norm[0, :])

angle_vectors = np.array([[np.cos(a / 180 * np.pi), np.sin(a / 180 * np.pi)] for a in mic_angles[mics]])
vectors = norm.dot(angle_vectors)
angles = np.arctan2(vectors[:, 1], vectors[:, 0]) * 180 / np.pi
weights = np.linalg.norm(vectors, axis=1)

significant = np.where(weights > 0.2)[0]

fig = plt.figure()
plt.plot(range(powers.shape[0]), angles, label=f"all")
plt.scatter(significant, angles[significant], label=f"significant")
fig.set_size_inches(10, 5)

fig = plt.figure()
plt.plot(range(powers.shape[0]), weights, label=f"mic{mic}")
fig.set_size_inches(10, 5)

In [ ]:
### statistics tests for alternatives to exponentiation in moving average

import scipy.stats
import scipy.signal
import matplotlib.pylab as plt
import numpy as np

scale1 = 1.0
scale2 = 4.0
loc1=0
loc2=3.0
step = 0.1
xmax = 30

n1 = scipy.stats.norm(loc=loc1, scale=scale1)
n2 = scipy.stats.norm(loc=loc2, scale=scale2)

scale_conv = np.sqrt(scale1**2 + scale2**2)
n_conv = scipy.stats.norm(loc=loc1 + loc2, scale=scale_conv)

values = np.arange(-xmax, xmax, step=step)
n1_pdf = n1.pdf(values)
n1_pdf /= np.sum(n1_pdf)
plt.plot(values, n1_pdf, label=f'N1')

n2_pdf = n2.pdf(values)
n2_pdf /= np.sum(n2_pdf)
plt.plot(values, n2_pdf, label=f'N2')

n_conv_pdf = n_conv.pdf(values)
n_conv_pdf /= np.sum(n_conv_pdf)
plt.plot(values, n_conv_pdf, label=f'analytical')

# compute new distribution through exponentiation. 
n2_new = scipy.stats.norm(loc1 + loc2, scale=scale1)
alpha = scale1**2 / (scale1**2 + scale2**2)

n2_approx_pdf = n2_new.pdf(values) ** alpha
n2_approx_pdf /= np.sum(n2_approx_pdf)
plt.plot(values, n2_approx_pdf, label=f'$N_1 ^ {alpha:.1f}$', ls=":", color='C0')

n2_conv = scipy.signal.fftconvolve(n1_pdf, n2_pdf, mode='same')
plt.plot(values, n2_conv, label=f'$N_1  N_2$', ls=":", color='C1')

plt.legend()

multimodal = np.zeros(len(values))
means = [-5, 0, 7.5]
stds = [1, 2, 3]
print(dict(zip(means, stds)))
plt.figure()
for mean, std in zip(means, stds):
    mode = scipy.stats.norm(loc=mean, scale=std)
    multimodal += mode.pdf(values)
    #plt.plot(values, vals, color='C2', ls=':')
multimodal /= np.sum(multimodal)
plt.plot(values, multimodal, color='C0', label='P1')
plt.plot(values, n2_pdf, color='C1', label='N2')
plt.legend(loc='upper right')

plt.figure()
# shift multimodal distribution
for scale in np.linspace(min(stds), max(stds), 3):
    alpha = scale**2 / (scale**2 + scale2**2)
    multimodal_approx_pdf = np.zeros_like(multimodal) 
    shift = int((loc2 - loc1) / step)
    if shift > 0:
        multimodal_approx_pdf[shift:] = multimodal[:int(len(multimodal) - shift)] ** alpha
    else:
        multimodal_approx_pdf[:int(len(multimodal) - shift)] = multimodal[shift:] ** alpha
    multimodal_approx_pdf /= np.sum(multimodal_approx_pdf)
    plt.plot(values, multimodal_approx_pdf, label=f'$P_1^{alpha:.1f}$ (std={scale:.1f})', ls="-")

multimodal_conv = scipy.signal.fftconvolve(multimodal, n2_pdf, mode='same')
multimodal_conv /= np.sum(multimodal_conv)
plt.plot(values, multimodal_conv, label=f'$P_1 N_2$', ls="-")
plt.legend(loc='upper right')

import timeit
print("time for convolution: ", end="")
print(round(timeit.timeit(stmt='scipy.signal.fftconvolve(multimodal, n2_pdf, mode="same")', globals=globals(), number=1000), 2))
print("time for exponentiation: ", end="")
print(round(timeit.timeit(stmt='multimodal_approx_pdf[shift:] = multimodal[:int(len(multimodal) - shift)] ** alpha', globals=globals(), number=1000), 2))